# Capstone Project - The Battle of Neighborhoods (Week 1)

Author: Dávid Kóczi

## Introduction <a name="introduction"></a>

In this project I will try to find an optimal location for a Chines restaurant in my country. Specifically, this report will be targeted to stakeholders interested in opening an **Chinese restaurant** in **Budapest**, Hungary.

Since there are lots of fast food restaurants in Budapest I will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Chinese restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.



### Table of content

* [Introduction:](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Chinese restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Budapest center will be obtained using **Google Maps API geocoding** of well known Budapest location

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 10x10 killometers centered around Budapest city center.

Let's first find the latitude & longitude of Budapest city center, using specific, well known address and Google Maps geocoding API.

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install geocoder

In [ ]:
pip install geopy

In [5]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install shapely

     |████████████████████████████████| 1.8MB 6.3MB/s eta 0:00:01███████████▎    | 1.6MB 6.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import sys
from bs4 import BeautifulSoup
import geocoder
import os
import folium # map rendering library
from geopy.geocoders import Nominatim 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import shapely.geometry
import pyproj
import pickle
import math

print('Libraries imported.')


Libraries imported.


In [34]:
foursquare_client_id = 'XXXXXXXXXX'
foursquare_client_secret ='XXXXXXXXXXX'
google_api_key = 'XXXXXXXXXXXX'

In [3]:
import requests


def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except Exception as e:
        print(e)
        return [None, None]
    
address = 'Palota,Budapes,Hungary'

budapest_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, budapest_center))

Coordinate of Palota,Budapes,Hungary: [47.4969189, 19.0361941]


In [10]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Budapest center longitude={}, latitude={}'.format(budapest_center[1], budapest_center[0]))
x, y = lonlat_to_xy(budapest_center[1], budapest_center[0])
print('Budapest center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Budapest center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Budapest center longitude=19.0361941, latitude=47.4969189
Budapest center UTM X=803975.6742963139, Y=5268287.058729421
Budapest center longitude=19.036194100000003, latitude=47.49691889999999


In [13]:
budapest_center_x, budapest_center_y = lonlat_to_xy(budapest_center[1], budapest_center[0]) # City center

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = budapest_center_x - 6000
x_step = 600
y_min = budapest_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(budapest_center_x, budapest_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [25]:
map_budapest = folium.Map(location=budapest_center, zoom_start=13)
folium.Marker(budapest_center, popup='Palota').add_to(map_budapest)
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_budapest) 
map_budapest

In [27]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, budapest_center[0], budapest_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(budapest_center[0], budapest_center[1], addr))

Reverse geocoding check
-----------------------
Address of [47.4969189, 19.0361941] is: Budapest, 1014 Budapest, Budavári Palota F épület, 1013 Hungary


In [28]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Hungary', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [29]:
addresses[150:170]

['Budapest, Vajda Péter u. 2, 1089 Hungary',
 'Budapest, Vajda Péter u. 8, 1089 Hungary',
 'Budapest, Kőbányai út 41, 1101 Hungary',
 'Budapest, Sötétvágás utca, 1121 Hungary',
 'Budapest, Hegyhát út 16, 1121 Hungary',
 'Budapest, Agancs út 17, 1121 Hungary',
 'Budapest, Széchenyi-emlék út 12b, 1121 Hungary',
 'Budapest, Hangya u. 42, 1121 Hungary',
 'Budapest, Tamási Áron u. 30, 1124 Hungary',
 'Budapest, Németvölgyi út 53c, 1124 Hungary',
 'Budapest, Kiss János altábornagy u. 33a, 1126 Hungary',
 'Budapest, Avar u. 5a, 1123 Hungary',
 'Budapest, Lisznyai u. 27, 1016 Hungary',
 'Budapest, Attila út 11, 1013 Hungary',
 'Budapest, Március 15. tér, 1056 Hungary',
 'Budapest, Henszlmann Imre u. 5, 1053 Hungary',
 'Budapest, Horánszky u. 21, 1085 Hungary',
 'Budapest, Őr u. 3, 1084 Hungary',
 'Budapest, Magdolna u. 22, 1086 Hungary',
 'Budapest, Orczy tér 1., 1087 Hungary']

In [30]:
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Budapest, Bedő u. 9, 1112 Hungary",47.446452,19.008445,802175.674296,5.262571e+06,5992.495307
1,"Budapest, Murányi u. 14, 1221 Hungary",47.446174,19.016384,802775.674296,5.262571e+06,5840.376700
2,"Budapest, Kubikos u. 3, 1116 Hungary",47.445895,19.024323,803375.674296,5.262571e+06,5747.173218
3,"Budapest, Verbéna u. 53, 1116 Hungary",47.445616,19.032262,803975.674296,5.262571e+06,5715.767665
4,"Budapest, Gyékényes u. 15, 1116 Hungary",47.445336,19.040200,804575.674296,5.262571e+06,5747.173218
5,"Budapest, Hunyadi János út 8, 1117 Hungary",47.445056,19.048139,805175.674296,5.262571e+06,5840.376700
6,"Budapest, Szikratávíró u. 12-14, 1211 Hungary",47.444774,19.056077,805775.674296,5.262571e+06,5992.495307
7,"Budapest, Egér út, 1112 Hungary",47.451533,18.996890,801275.674296,5.263091e+06,5855.766389
8,"Budapest, Repülőtéri út 6, 1112 Hungary",47.451255,19.004830,801875.674296,5.263091e+06,5604.462508
9,"Budapest, Ütköző sor 12, 1112 Hungary",47.450977,19.012770,802475.674296,5.263091e+06,5408.326913


In [31]:
df_locations.to_pickle('./locations.pkl')    

In [36]:
# Category IDs corresponding to Chinies restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

Chines_restaurant_categories = ['4bf58dd8d48988d145941735','52af3a5e3cf9994f4e043bea','52af3a723cf9994f4e043bec',
                                 '52af3a7c3cf9994f4e043bed','58daa1558bbb0b01f18ec1d3','52af3a673cf9994f4e043beb',
                                 '52af3a903cf9994f4e043bee','4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef',
                                 '52af3aaa3cf9994f4e043bf0','52af3ab53cf9994f4e043bf1','52af3abe3cf9994f4e043bf2',
                                 '52af3ac83cf9994f4e043bf3','52af3ad23cf9994f4e043bf4','52af3add3cf9994f4e043bf5',
                                 '52af3af23cf9994f4e043bf7','52af3ae63cf9994f4e043bf6','52af3afc3cf9994f4e043bf8',
                                 '52af3b053cf9994f4e043bf9','52af3b213cf9994f4e043bfa','52af3b293cf9994f4e043bfb',
                                 '52af3b343cf9994f4e043bfc','52af3b3b3cf9994f4e043bfd','52af3b463cf9994f4e043bfe',
                                 '52af3b6e3cf9994f4e043c02','52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04',
                                 '52af3b893cf9994f4e043c05','52af3b913cf9994f4e043c06','52af3b9a3cf9994f4e043c07',                                 
                                 '52af3b633cf9994f4e043c01','52af3b513cf9994f4e043bff','52af3b593cf9994f4e043c00',
                                 '52af3ba23cf9994f4e043c08']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Hungary', '')
    address = address.replace(', Budapest', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [38]:
def get_restaurants(lats, lons):
    restaurants = {}
    Chines_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_Chines = is_restaurant(venue_categories, specific_filter=Chines_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_Chines, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_Chines:
                    Chines_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, Chines_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
Chines_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('Chines_restaurants_350.pkl', 'rb') as f:
        Chines_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, Chines_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('Chines_restaurants_350.pkl', 'wb') as f:
        pickle.dump(Chines_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [40]:
print('Total number of restaurants:', len(restaurants))
print('Total number of Chines restaurants:', len(Chines_restaurants))
print('Percentage of Chines restaurants: {:.2f}%'.format(len(Chines_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1195
Total number of Chines restaurants: 107
Percentage of Chines restaurants: 8.95%
Average number of restaurants in neighborhood: 2.9148351648351647


In [41]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('51cb0f1f498ec6821e1b8ddf', 'Regős Vendéglő', 47.44873665124521, 19.00875657477026, 'Budapest, Kapolcs Utca 2/a, Magyarország', 255, False, 802186.0789610844, 5262826.310361378)
('5abe23db86f4cc283628c3bc', 'Rétisas Étterem', 47.447865, 19.025368, 'Budapest, 1116, Magyarország', 232, False, 803443.1180816896, 5262794.231572624)
('4d7cd7fdcf3f37046c635840', 'Két Dió Vendéglő', 47.44807715976893, 19.025110176387784, 'Budapest, Hunyadi Mátyás út 56, 1116, Magyarország', 250, False, 803422.4633907927, 5262816.797640705)
('4e143bb562e14518a923543e', 'Mazsola Konyhája', 47.443572805589206, 19.04454422207084, 'Budapest, Vegyész u. 17-25., 1116, Magyarország', 316, False, 804913.3094058295, 5262392.4263895685)
('5a02bd4c6e465055da6b9cfc', 'Péterhegyi Lejtő', 47.45127, 19.016428, 'Budapest, 1112, Magyarország', 328, False, 802749.7212301941, 5263137.674752849)
('4bd17410caff9521d0aad0f0', 'VakVarjú Vendéglő és Sörkert', 47.449675522614854, 19.036

In [42]:
print('List of Chines restaurants')
print('---------------------------')
for r in list(Chines_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(Chines_restaurants))

List of Chines restaurants
---------------------------
('4d1b1d5c6526a35d70d00c16', 'Renhe Kínai Gyors Büfé', 47.46020340279991, 19.05195526708575, 'Budapest, Újbuda Center (Budafoki út), 1117, Magyarország', 171, True, 805375.5050742112, 5264269.399003791)
('4e9ea27330f8fb677600b52d', 'Yu Cheng Kínai Gyorsbüfé', 47.465346722888484, 19.022654357379107, 'Budapest, Etele tér 5., 1115, Magyarország', 249, True, 803137.9426323414, 5264726.058479761)
('4ebd23f229c2a4fb0a2cc93f', 'Kínai Büfé', 47.463836, 19.033225, 'Magyarország', 59, True, 803943.1299851767, 5264599.526640626)
('4d3055cd789a8cfa530a2ec6', 'Kínai étterem', 47.46287984902365, 19.04245079055973, 'Budapest, Fehérvári út 88/b (Hengermalom út), 1119, Magyarország', 138, True, 804643.81710959, 5264529.445585362)
('507a910de4b0c101c6e15467', 'Hong Li Kínai Gyorsétterem', 47.46596957328054, 19.043284187397806, 'Magyarország', 347, True, 804688.7325836045, 5264876.020426936)
('4d9b02865e52224b2a6038e3', 'Kínai Büfé', 47.4713676630564

In [43]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Talponálló kifőzde
Restaurants around location 102: Kakukkfészek
Restaurants around location 103: 
Restaurants around location 104: Bécsiszelet Vendéglő
Restaurants around location 105: Mermel étterem, Kuriózum Café
Restaurants around location 106: Hemingway Étterem, Kisvigadó, Montenegroi Gurman, Family Restaurant Kifőzde, Gyros Büfé, Corner Falatozó
Restaurants around location 107: Deli's Vegán Bisztró, Eker Gyros Török Étterem, Las Vegans, Buono by Il Treno, Istanbul Kebabs Non-Stop, Madárfészek gyorsétterem és kávézó, Madárfészek Kínai Gyorsétterem, Döner kebab
Restaurants around location 108: Gyros, El Vandom salátabár, Pho 74, Íz-lelő Étkezde, Ganesha Vega, Stoczek Menza, Bercsényi Kisvendéglő, Rondo Gyorsétterem és Kávézó, Ji Li Kínai Büfé
Restaurants around location 109: A38 Étterem
Restaurants around location 110: Saloniki Gyros Taverna, Wok Express, Hange Restaurant (Hange Étterem), Lele 

In [45]:
map_budapest = folium.Map(location=budapest_center, zoom_start=13)
folium.Marker(budapest_center, popup='Palota').add_to(map_budapest)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_budapest = res[6]
    color = 'red' if is_budapest else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_budapest)
map_budapest

Now we have all the restaurants in area within few kilometers from Budapest Palota , and we know which ones are Chines restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Chines restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Budapest that have low restaurant density, particularly those with low number of Chines restaurants. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from Budapest center** (Budapest Palota). We have also **identified Chines restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Budapest - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Chines restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Chines restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

WEEK 2

## Results and Discussion <a name="results"></a>

Week 2

## Conclusion <a name="conclusion"></a>

This project prepared by based on the example project. Thank you for your review!

Week 2